# Image2Image Pipeline for Stable Diffusion using 🧨 Diffusers 

This notebook shows how to create a custom `diffusers` pipeline for  text-guided image-to-image generation with Stable Diffusion model using  🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers). 

For a general introduction to the Stable Diffusion model please refer to this [colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).



# Setup

## import GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Check GPU

In [ ]:
!nvidia-smi

## Install libs

In [ ]:
!pip install diffusers==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"

You need to accept the model license before downloading or using the weights. In this post we'll use model version `v1-4`, so you'll need to  visit [its card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. 

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

# Token Login

In [ ]:
from huggingface_hub import notebook_login
import inspect
import warnings
from typing import List, Optional, Union
import torch
from torch import autocast
from tqdm.auto import tqdm
import requests
from io import BytesIO
from PIL import Image
from datetime import datetime
from diffusers import StableDiffusionImg2ImgPipeline

notebook_login()

## Image2Image pipeline.

Load the pipeline

In [ ]:
device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
)
pipe = pipe.to(device)

Download an initial image and preprocess it so we can pass it to the pipeline.

In [ ]:
batch_init_path = "/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/batch_mode/"

In [ ]:
def produce_one_image_by_params(seed, init_image_path, prompt, s_ratio):
  
  init_img = Image.open(init_image_path).convert("RGB")
  init_img = init_img.resize((512, 512))

  ### black and white option
  # init_img = init_img.convert('L')
  # init_img = init_img.convert('RGB')

  generator = torch.Generator(device=device).manual_seed(seed)

  now = datetime.now() 
  time = now.strftime("%H%M%S")

  with autocast("cuda"):
      pipe.safety_checker = (lambda images, clip_input: (images, False))
      image = pipe(prompt=prompt, init_image=init_img, strength=s_ratio, guidance_scale=7.5, generator=generator).images[0]
  image.save('/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/'+time+'image.png')
  display(image)

In [ ]:
prompt = "oil on canvas portrait of a 20 year old bearded man in the style of Grant Wood, trending on Artstation"

In [ ]:
from glob import glob


extension = 'png'
# extension = 'jpg'
input_file_paths = glob(batch_init_path+"*/*."+extension, recursive = True)

# for seed in range(0, 100, 1):
seed = 42
for file_path in input_file_paths:
  for s_ratio in range(30, 80, 5):
    for curr_seed in range(1, 4, 1):
      produce_one_image_by_params(curr_seed*42, file_path, prompt, s_ratio/100)

In [ ]:
!mkdir /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S) && mv /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/*.png /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S)